TODO: 

I) version base
- (done)  reprendre classifieur de chat hiérarchique, remplacer encodeur seq2vec par un seq2seq (stacked lstm eg)
    cf HierarchicalChatSequenceClassification
- (done)  ajouter séquence de labels / comme si arbre ou forêt (possible ds graph_parser ?)

- sousclasser graph_parser / modifs
    plus facile en forcant label unique sur les arcs ? à voir
- brancher données réelles

/home/muller/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/data/dataset_readers/semantic_dependency_parsing.py


améliorations: 
- encodeur tour -> bert
- graphe de labels -> AdjacencyMatrixField


In [1]:
import itertools
import logging
from typing import Any, Tuple, Dict, List, Iterable
import torch
import torch.optim as optim

In [2]:
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, LabelField, TextField, ListField, SequenceLabelField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer, PretrainedTransformerTokenizer
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding

from allennlp.modules import Seq2VecEncoder, Seq2SeqEncoder
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.data.vocabulary import Vocabulary

from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator
from allennlp.common import Params

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from dummy_chat_reader import ChatReader

In [5]:
token_indexers = {"tokens": SingleIdTokenIndexer()}

tokenizer_cfg = Params({"word_splitter": {"language": "en"}})

tokenizer = Tokenizer.from_params(tokenizer_cfg)


reader = ChatReader(
    tokenizer=tokenizer,
    token_indexers=token_indexers,
    )
train_instances = reader.read("./train_dummy.tsv")
vocab = Vocabulary.from_instances(train_instances)


for i in train_instances:
    print(i)
    i["lines"].index(vocab)
    i["label"].index(vocab)
    print(i["lines"].get_padding_lengths())

2it [00:00, 1033.08it/s]
100%|██████████| 2/2 [00:00<00:00, 20164.92it/s]

Instance with fields:
 	 lines: ListField of 3 TextFields : 
 	 TextField of length 7 with text: 
 		[this, is, a, turn, in, a, chat]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 2 with text: 
 		[another, one]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[but, this, is, different]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 
 	 label: AdjacencyField of length 3
		with indices:
 		[(0, 2), (1, 2), (2, 2)]

		and labels:
 		None
 		in namespace: 'labels'. 

{'num_fields': 3, 'list_num_tokens': 7, 'list_tokens_length': 7}
Instance with fields:
 	 lines: ListField of 4 TextFields : 
 	 TextField of length 3 with text: 
 		[another, chat, starts]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[is, it, different, ?]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[well, a, little, bit

In [6]:
print(i)

Instance with fields:
 	 lines: ListField of 4 TextFields : 
 	 TextField of length 3 with text: 
 		[another, chat, starts]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[is, it, different, ?]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[well, a, little, bit]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 9 with text: 
 		[but, we, need, a, different, number, of, turns, .]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 
 	 label: AdjacencyField of length 4
		with indices:
 		[(0, 3), (1, 3), (2, 3), (3, 3)]

		and labels:
 		None
 		in namespace: 'labels'. 



In [7]:
turn_encoder_cfg = Params({"type":"gru",'input_size': 100, 'hidden_size': 50, 'num_layers': 1,
                  'dropout': 0.25, 'bidirectional': False
})
#can be changed dynamically encoder_cfg["type"] = "lstm"
# warning: if bidirectional, state output dimension is hidden_size x 2 -> model doesn't know that

turn_encoder = Seq2VecEncoder.from_params(turn_encoder_cfg)
turn_encoder.hidden_size = turn_encoder_cfg["hidden_size"]


chat_encoder_cfg = Params({"type":"gru",'input_size': 50, 'hidden_size': 50, 'num_layers': 1,
                  'dropout': 0.25, 'bidirectional': False
})
chat_encoder = Seq2SeqEncoder.from_params(chat_encoder_cfg)
chat_encoder.hidden_size = chat_encoder_cfg["hidden_size"]



glove_text_field_embedder = Embedding.from_params(vocab,Params({"pretrained_file": "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.100d.txt.gz",
                                                          "embedding_dim": 100,
                                                          "trainable": False
}))

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=100)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})



/home/muller/anaconda3/envs/allennlp/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
400000it [00:02, 161796.65it/s]


In [8]:
chat_encoder

PytorchSeq2SeqWrapper(
  (_module): GRU(50, 50, batch_first=True, dropout=0.25)
)

In [9]:
# from biaffine parser, another config (not used)
chat_encoder_cfg =  {
            "type": "stacked_bidirectional_lstm",
            "hidden_size": 400,
            "input_size": 200,
            "num_layers": 3,
            "recurrent_dropout_probability": 0.3,
            "use_highway": True
        }

In [ ]:
from allennlp.models import Model
from typing import Dict, List, Iterable
from allennlp.modules import TimeDistributed
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

In [11]:
from allennlp.models import Model
from allennlp.common.checks import check_dimensions_match, ConfigurationError
from allennlp.data import Vocabulary
from allennlp.modules import Seq2SeqEncoder, TextFieldEmbedder, Embedding, InputVariationalDropout
from allennlp.modules.matrix_attention.bilinear_matrix_attention import BilinearMatrixAttention
from allennlp.modules import FeedForward
from allennlp.models.model import Model
from allennlp.nn import InitializerApplicator, Activation
#???? TODO from allennlp.nn.util import min_value_of_dtype -> only allennlp >= 1.0
def min_value_of_dtype(dtype: torch.dtype):
    """
    Returns the minimum value of a given PyTorch data type. Does not allow torch.bool.
    """
    return info_value_of_dtype(dtype).min
def info_value_of_dtype(dtype: torch.dtype):
    """
    Returns the `finfo` or `iinfo` object of a given PyTorch data type. Does not allow torch.bool.
    """
    if dtype == torch.bool:
        raise TypeError("Does not support torch.bool")
    elif dtype.is_floating_point:
        return torch.finfo(dtype)
    else:
        return torch.iinfo(dtype)


from allennlp.nn.util import get_text_field_mask
from allennlp.nn.util import get_lengths_from_binary_sequence_mask
from allennlp.training.metrics import F1Measure

import copy
from overrides import overrides
import torch
from torch.nn.modules import Dropout
import numpy

class ChatGraphParser(Model):
    """
    A Parser for arbitrary graph structures.

    Registered as a `Model` with name "graph_parser".

    # Parameters

    vocab : `Vocabulary`, required
        A Vocabulary, required in order to compute sizes for input/output projections.
    text_field_embedder : `TextFieldEmbedder`, required
        Used to embed the `tokens` `TextField` we get as input to the model.
    encoder : `Seq2SeqEncoder`
        The encoder (with its own internal stacking) that we will use to generate representations
        of tokens.
    tag_representation_dim : `int`, required.
        The dimension of the MLPs used for arc tag prediction.
    arc_representation_dim : `int`, required.
        The dimension of the MLPs used for arc prediction.
    tag_feedforward : `FeedForward`, optional, (default = None).
        The feedforward network used to produce tag representations.
        By default, a 1 layer feedforward network with an elu activation is used.
    arc_feedforward : `FeedForward`, optional, (default = None).
        The feedforward network used to produce arc representations.
        By default, a 1 layer feedforward network with an elu activation is used.
    pos_tag_embedding : `Embedding`, optional.
        Used to embed the `pos_tags` `SequenceLabelField` we get as input to the model.
    dropout : `float`, optional, (default = 0.0)
        The variational dropout applied to the output of the encoder and MLP layers.
    input_dropout : `float`, optional, (default = 0.0)
        The dropout applied to the embedded text input.
    edge_prediction_threshold : `int`, optional (default = 0.5)
        The probability at which to consider a scored edge to be 'present'
        in the decoded graph. Must be between 0 and 1.
    initializer : `InitializerApplicator`, optional (default=`InitializerApplicator()`)
        Used to initialize the model parameters.
    """

    def __init__(
        self,
        vocab: Vocabulary,
        text_field_embedder: TextFieldEmbedder,
        turn_encoder: Seq2VecEncoder, 
        chat_encoder: Seq2SeqEncoder,
        arc_representation_dim: int,
        arc_feedforward: FeedForward = None,
        dropout: float = 0.0,
        input_dropout: float = 0.0,
        edge_prediction_threshold: float = 0.5,
        initializer: InitializerApplicator = InitializerApplicator(),
        **kwargs,
    ) -> None:
        super().__init__(vocab, **kwargs)
        
        self.text_field_embedder = text_field_embedder
        self.turn_encoder = TimeDistributed(turn_encoder)
        self.chat_encoder = chat_encoder
        
        self.edge_prediction_threshold = edge_prediction_threshold
        if not 0 < edge_prediction_threshold < 1:
            raise ConfigurationError(
                f"edge_prediction_threshold must be between "
                f"0 and 1 (exclusive) but found {edge_prediction_threshold}."
            )

        encoder_dim = chat_encoder.get_output_dim()

        self.head_arc_feedforward = arc_feedforward or FeedForward(
            encoder_dim, 1, arc_representation_dim, Activation.by_name("elu")()
        )
        self.child_arc_feedforward = copy.deepcopy(self.head_arc_feedforward)

        self.arc_attention = BilinearMatrixAttention(
            arc_representation_dim, arc_representation_dim, use_input_biases=True
        )

        self._dropout = InputVariationalDropout(dropout)
        self._input_dropout = Dropout(input_dropout)

        representation_dim = turn_encoder.get_output_dim()

        self._unlabelled_f1 = F1Measure(positive_label=1)
        self._arc_loss = torch.nn.BCEWithLogitsLoss(reduction="none")
        initializer(self)
        # useful for debugging
        self.iter_count = 0 
        
    # init done
        
    # todo 
    @overrides
    def forward(
        self,  # type: ignore
        lines,
        label: torch.LongTensor = None,
        metadata: List[Dict[str, Any]] = None,
    ) -> Dict[str, torch.Tensor]:

        """
        # Parameters

        lines: the chat as a list of turns, each being a list of token
        TODO: add metadata to instances
        metadata : List[Dict[str, Any]], optional (default = None)
            A dictionary of metadata for each batch element which has keys:
                tokens : `List[str]`, required.
                    The original string tokens in the sentence.
        label : a tensor containing the adjacency matrix for the instance dependencies between turns
            Has shape `(batch_size, sequence_length, sequence_length)`.

        # Returns

        An output dictionary.
        """
        #########
        # this is the part where chat is encoded as sequence of turn encodings
        #########
        # mask for each turn of each chat of the batch: shape = (batch_size x max_turns x tokens)
        token_mask = get_text_field_mask(lines,num_wrapping_dims=1)

        # chat turns fetching embedding
        # turns_embedding tensor is (batch_size x turns x max tokens x token embedding size)
        turns_embeddings = self.text_field_embedder(lines,num_wrapping_dims=1)
      
        # encoding turns
        # turn_h has shape (batch_size x turns x encoder_output_size) 
        turn_h = self.turn_encoder(turns_embeddings,token_mask)
        
        # mask for chats is now nb of turns; beware weird return type of torch.max (tuple) 
        chat_mask = token_mask.max(axis=2)[0]
        
        # renaming to mask -> easier to transpose the rest of graph_parser
        mask = chat_mask
        
        # graph parser goes on
        # leave input dropout for now
        # embedded_text_input = turn_h equivalent in hierarchical sequence -> renaming 
        #embedded_text_input = self._input_dropout(embedded_text_input)
        embedded_text_input = turn_h
        # we keep graph parser original name for now
        # encoded_text = encoded chat = self.chat_encoder(turn_h,chat_mask) equivalent in hierarchical sequence
        encoded_text = self.chat_encoder(embedded_text_input, mask)

        encoded_text = self._dropout(encoded_text)

        # shape (batch_size, sequence_length, arc_representation_dim)
        head_arc_representation = self._dropout(self.head_arc_feedforward(encoded_text))
        child_arc_representation = self._dropout(self.child_arc_feedforward(encoded_text))

        # shape (batch_size, sequence_length, tag_representation_dim)
        #head_tag_representation = self._dropout(self.head_tag_feedforward(encoded_text))
        #child_tag_representation = self._dropout(self.child_tag_feedforward(encoded_text))
        
        # shape (batch_size, sequence_length, sequence_length)
        arc_scores = self.arc_attention(head_arc_representation, child_arc_representation)
        
        # shape (batch_size, num_tags, sequence_length, sequence_length)
        #arc_tag_logits = self.tag_bilinear(head_tag_representation, child_tag_representation)
        # Switch to (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_logits = arc_tag_logits.permute(0, 2, 3, 1).contiguous()

        # Since we'll be doing some additions, using the min value will cause underflow
        # CHAT: unncessary since we dont have a loss for labels
        #minus_mask = ~mask * min_value_of_dtype(arc_scores.dtype) / 10
        #arc_scores = arc_scores + minus_mask.unsqueeze(2) + minus_mask.unsqueeze(1)

        
        #breakpoint()
        arc_probs = self._greedy_decode(arc_scores, mask)

        output_dict = {"arc_probs": arc_probs, "mask": mask}

        if metadata:
            output_dict["tokens"] = [meta["tokens"] for meta in metadata]

        arc_tags = label # gold labels -> here just the adjacency matrix 0/1 ? 
        if arc_tags is not None:
            arc_nll= self._construct_loss(
                arc_scores=arc_scores, arc_tags=arc_tags, mask=mask
            )
            # same here with no arc relations ; keep all anyway to prevent ubgs downstream (TODO: coherent renaming)
            output_dict["loss"] = arc_nll 
            output_dict["arc_loss"] = arc_nll
        

            # Make the arc tags not have negative values anywhere
            # (by default, no edge is indicated with -1).
            # NB re chat: probably not useful, but kept as a precaution
            arc_indices = (arc_tags != -1).float()
            tag_mask = mask.unsqueeze(1) & mask.unsqueeze(2)
            one_minus_arc_probs = 1 - arc_probs
            # We stack scores here because the f1 measure expects a
            # distribution, rather than a single value.
            self._unlabelled_f1(
                torch.stack([one_minus_arc_probs, arc_probs], -1), arc_indices, tag_mask
            )
        self.iter_count += 1
        if False and self.get_metrics()["f1"]>=0.6:
            breakpoint()
        return output_dict
    # modified / untested
    
    def _construct_loss(
        self,
        arc_scores: torch.Tensor,
        arc_tags: torch.Tensor,
        mask: torch.BoolTensor,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Computes the arc and tag loss for an adjacency matrix.

        # Parameters

        arc_scores : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length) used to generate a
            binary classification decision for whether an edge is present between two words.
        #####arc_tag_logits : `torch.Tensor`, required.
        #####    A tensor of shape (batch_size, sequence_length, sequence_length, num_tags) used to generate
        #####    a distribution over edge tags for a given edge.
        arc_tags : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length).
            The labels for every arc.
        mask : `torch.BoolTensor`, required.
            A mask of shape (batch_size, sequence_length), denoting unpadded
            elements in the sequence.

        # Returns

        arc_nll : `torch.Tensor`, required.
            The negative log likelihood from the arc loss.
        tag_nll : `torch.Tensor`, required.
            The negative log likelihood from the arc tag loss.
        """
        arc_indices = (arc_tags != -1).float()
        # Make the arc tags not have negative values anywhere
        # (by default, no edge is indicated with -1).
        arc_tags = arc_tags * arc_indices
        arc_nll = self._arc_loss(arc_scores, arc_indices) * mask.unsqueeze(1) * mask.unsqueeze(2)
        # We want the mask for the tags to only include the unmasked words
        # and we only care about the loss with respect to the gold arcs.
        tag_mask = mask.unsqueeze(1) * mask.unsqueeze(2) * arc_indices

        #batch_size, sequence_length, _, num_tags = arc_tag_logits.size()
        #original_shape = [batch_size, sequence_length, sequence_length]
        #reshaped_logits = arc_tag_logits.view(-1, num_tags)
        reshaped_tags = arc_tags.view(-1)
        #tag_nll = (
        #    self._tag_loss(reshaped_logits, reshaped_tags.long()).view(original_shape) * tag_mask
        #)

        valid_positions = tag_mask.sum()

        arc_nll = arc_nll.sum() / valid_positions.float()
        #tag_nll = tag_nll.sum() / valid_positions.float()
        return arc_nll#, tag_nll
    # modified/untested

    @staticmethod
    def _greedy_decode(
        arc_scores: torch.Tensor, mask: torch.BoolTensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Decodes the head and head tag predictions by decoding the unlabeled arcs
        independently for each word and then again, predicting the head tags of
        these greedily chosen arcs independently.

        # Parameters

        arc_scores : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length) used to generate
            a distribution over attachments of a given word to all other words.
        ###arc_tag_logits : `torch.Tensor`, required.
        ###    A tensor of shape (batch_size, sequence_length, sequence_length, num_tags) used to
        ###    generate a distribution over tags for each arc.
        mask : `torch.BoolTensor`, required.
            A mask of shape (batch_size, sequence_length).

        # Returns

        arc_probs : `torch.Tensor`
            A tensor of shape (batch_size, sequence_length, sequence_length) representing the
            probability of an arc being present for this edge.
        ####arc_tag_probs : `torch.Tensor`
        ####    A tensor of shape (batch_size, sequence_length, sequence_length, sequence_length)
        ####    representing the distribution over edge tags for a given edge.
        """
        # Mask the diagonal, because we don't self edges.
        # WARNING: might not be the case for chats ? 
        inf_diagonal_mask = torch.diag(arc_scores.new(mask.size(1)).fill_(-numpy.inf))
        arc_scores = arc_scores + inf_diagonal_mask
        # shape (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_logits = arc_tag_logits + inf_diagonal_mask.unsqueeze(0).unsqueeze(-1)
        # Mask padded tokens, because we only want to consider actual word -> word edges.
        # CHAT: this is the wrong torch version lol this does not work/ confusion int/bools
        # minus_mask = ~mask.unsqueeze(2)
        # CHAT: this should work with torch>1.4
        minus_mask = (mask<1).unsqueeze(2)
        
        arc_scores.masked_fill_(minus_mask, -numpy.inf)
        # but shouldn'it mask both padded lines/columns ? just does lines
        
        #arc_tag_logits.masked_fill_(minus_mask.unsqueeze(-1), -numpy.inf)
        # shape (batch_size, sequence_length, sequence_length)
        arc_probs = arc_scores.sigmoid()
        # shape (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_probs = torch.nn.functional.softmax(arc_tag_logits, dim=-1)
        return arc_probs#, arc_tag_probs
    # modified / untested
    
    @overrides
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        metrics = {}
        precision, recall, f1_measure = self._unlabelled_f1.get_metric(reset)
        metrics["precision"] = precision
        metrics["recall"] = recall
        metrics["f1"] = f1_measure
        return metrics

 

In [36]:
#model = HierarchicalChatSequenceClassification(vocab,word_embeddings,turn_encoder,chat_encoder)



train_instances = reader.read("./train_dummy.tsv")
vocab = Vocabulary.from_instances(train_instances)


arc_representation_dim = 50 
model = ChatGraphParser(vocab,word_embeddings,
                        turn_encoder,chat_encoder,arc_representation_dim,
                       edge_prediction_threshold=0.5)


from allennlp.training.optimizers import Optimizer 
trainer_cfg = {
        "cuda_device": -1,
        "grad_norm": 5,
        "num_epochs": 100,
        "optimizer": {
            "type": "dense_sparse_adam",
            "betas": [
                0.9,
                0.9
            ]
        },
        "patience": 50,
}


opt_cfg = trainer_cfg.pop("optimizer")


optimizer = optim.SGD(model.parameters(), lr=0.1)
#does not work cos needs model parameters too
#optimizer = Optimizer.from_params(opt_cfg)

iterator = BucketIterator(batch_size=2,sorting_keys=[("lines","list_num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  num_epochs=500,
                  patience=20,
                  train_dataset=train_instances,
                  should_log_parameter_statistics = False
)

2it [00:00, 1406.30it/s]
100%|██████████| 2/2 [00:00<00:00, 21509.25it/s]


In [37]:
trainer.train()

precision: 0.0000, recall: 0.0000, f1: 0.0000, loss: 2.3443 ||: 100%|██████████| 1/1 [00:00<00:00, 117.74it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.5348 ||: 100%|██████████| 1/1 [00:00<00:00, 86.81it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0375 ||: 100%|██████████| 1/1 [00:00<00:00, 98.01it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0216 ||: 100%|██████████| 1/1 [00:00<00:00, 85.31it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0157 ||: 100%|██████████| 1/1 [00:00<00:00, 94.60it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0125 ||: 100%|██████████| 1/1 [00:00<00:00, 79.61it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0104 ||: 100%|██████████| 1/1 [00:00<00:00, 92.46it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0090 ||: 100%|██████████| 1/1 [00:00<00:00, 76.52it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0079 ||: 100%|██████████| 1/1 [00:00<00:00, 122.94it/s]
precisio

precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0010 ||: 100%|██████████| 1/1 [00:00<00:00, 107.49it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0010 ||: 100%|██████████| 1/1 [00:00<00:00, 110.29it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0010 ||: 100%|██████████| 1/1 [00:00<00:00, 117.96it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0010 ||: 100%|██████████| 1/1 [00:00<00:00, 126.17it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0009 ||: 100%|██████████| 1/1 [00:00<00:00, 104.48it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0009 ||: 100%|██████████| 1/1 [00:00<00:00, 112.08it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0009 ||: 100%|██████████| 1/1 [00:00<00:00, 112.51it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0009 ||: 100%|██████████| 1/1 [00:00<00:00, 93.81it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0009 ||: 100%|██████████| 1/1 [00:00<00:00, 102.84it/s]
pr

precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 103.19it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 110.01it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 102.41it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 92.29it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 106.56it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 77.90it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 100.09it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 111.09it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 108.21it/s]
pre

precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 115.42it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 97.25it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 104.29it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 93.99it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 94.03it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 92.08it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 91.81it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 87.11it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0003 ||: 100%|██████████| 1/1 [00:00<00:00, 98.17it/s]
precisio

precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 106.84it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 97.48it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 92.64it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 107.92it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 76.56it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 98.32it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 96.81it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 103.33it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 106.67it/s]
precis

precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 101.45it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 103.99it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 86.94it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 104.47it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 92.71it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 92.61it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 95.42it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 78.67it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0002 ||: 100%|██████████| 1/1 [00:00<00:00, 111.04it/s]
precis

precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 102.85it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 96.02it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 117.06it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 94.30it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 96.15it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 95.90it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 97.97it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 90.00it/s]
precision: 1.0000, recall: 0.7143, f1: 0.8333, loss: 0.0001 ||: 100%|██████████| 1/1 [00:00<00:00, 97.20it/s]
precisio

{'best_epoch': 499,
 'peak_cpu_memory_MB': 334.824,
 'peak_gpu_0_memory_MB': 960,
 'training_duration': '0:00:15.471880',
 'training_start_epoch': 0,
 'training_epochs': 499,
 'epoch': 499,
 'training_precision': 1.0,
 'training_recall': 0.7142857313156128,
 'training_f1': 0.8333333730697632,
 'training_loss': 0.0001285533217014745,
 'training_cpu_memory_MB': 334.824,
 'training_gpu_0_memory_MB': 960}

In [25]:
opt_cfg

{'type': 'dense_sparse_adam', 'betas': [0.9, 0.9]}

In [14]:
vocab

Vocabulary with namespaces:  tokens, Size: 24 || Non Padded Namespaces: {'*labels', '*tags'}

In [15]:
mask = torch.tensor([[1, 1, 1, 0],
        [1, 1, 1, 1]])

In [16]:
inf = numpy.inf 
arc_scores = torch.tensor([[[  -inf, 0.1252, 0.1161, 0.0475],
         [0.0977,   -inf, 0.1015, 0.0373],
         [0.1541, 0.1673,   -inf, 0.0934],
         [0.0563, 0.0730, 0.0623,   -inf]],

        [[  -inf, 0.0947, 0.1048, 0.1114],
         [0.1385,   -inf, 0.1681, 0.1755],
         [0.1114, 0.1310,   -inf, 0.1448],
         [0.1274, 0.1454, 0.1537,   -inf]]])

In [17]:
minus_mask = (mask<1).unsqueeze(2)    
arc_scores.masked_fill_(minus_mask, -numpy.inf)         

tensor([[[  -inf, 0.1252, 0.1161, 0.0475],
         [0.0977,   -inf, 0.1015, 0.0373],
         [0.1541, 0.1673,   -inf, 0.0934],
         [  -inf,   -inf,   -inf,   -inf]],

        [[  -inf, 0.0947, 0.1048, 0.1114],
         [0.1385,   -inf, 0.1681, 0.1755],
         [0.1114, 0.1310,   -inf, 0.1448],
         [0.1274, 0.1454, 0.1537,   -inf]]])

In [18]:
arc_probs = arc_scores.sigmoid()    

In [19]:
mask

tensor([[1, 1, 1, 0],
        [1, 1, 1, 1]])

In [20]:
minus_mask = (mask.unsqueeze(1) & mask.unsqueeze(2))<1